In [33]:
import pandas as pd
import numpy as np 


In [35]:
df = pd.read_csv("covid_19_prediction/data/covid19.csv", index_col=0, sep=",")
df.head(10)

,dayFromStart,day,month,LJ regija,po regija,kr regija,ce regija,foreign regija,nm regija,kk regija,...,age 55-64 MEN,age 65-74,age 65-74 FEMALE,age 65-74 MEN,age 75-84,age 75-84 FEMALE,age 75-84 MEN,age 85+,age 85+ FEMALE,age 85+ MEN
year,,,,,,,,,,,,,,,,,,,,,
2020,1,4,3,1,NaN,NaN,NaN,NaN,NaN,NaN,...,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2020,2,5,3,5,NaN,NaN,NaN,NaN,NaN,NaN,...,1,1.0,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN
2020,3,6,3,7,NaN,NaN,NaN,NaN,1.0,NaN,...,3,1.0,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN
2020,4,7,3,8,NaN,NaN,NaN,NaN,3.0,NaN,...,3,2.0,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN
2020,5,8,3,10,NaN,NaN,NaN,NaN,6.0,NaN,...,4,3.0,1.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN
2020,6,9,3,16,NaN,NaN,NaN,NaN,11.0,NaN,...,5,5.0,1.0,4.0,NaN,NaN,NaN,1.0,1.0,NaN
2020,7,10,3,27,2.0,NaN,NaN,NaN,12.0,NaN,...,7,6.0,1.0,5.0,NaN,NaN,NaN,2.0,1.0,1.0
2020,8,11,3,44,3.0,1.0,4.0,NaN,18.0,NaN,...,9,8.0,2.0,6.0,1.0,1.0,NaN,3.0,1.0,2.0
2020,9,12,3,63,3.0,2.0,13.0,1.0,30.0,1.0,...,15,8.0,2.0,6.0,1.0,1.0,NaN,5.0,2.0,3.0


In [46]:
df.tail(10)

,dayFromStart,day,month,LJ regija,po regija,kr regija,ce regija,foreign regija,nm regija,kk regija,...,age 55-64 MEN,age 65-74,age 65-74 FEMALE,age 65-74 MEN,age 75-84,age 75-84 FEMALE,age 75-84 MEN,age 85+,age 85+ FEMALE,age 85+ MEN
year,,,,,,,,,,,,,,,,,,,,,
2020,294,22,12,26274,2326.0,12784.0,15208.0,482.0,8560.0,4011.0,...,7518,8003.0,3693.0,4310.0,7368.0,4382.0,2986.0,6581.0,5107.0,1474.0
2020,295,23,12,26673,2359.0,12932.0,15556.0,487.0,8771.0,4144.0,...,7695,8185.0,3781.0,4404.0,7479.0,4441.0,3038.0,6648.0,5165.0,1483.0
2020,296,24,12,27064,2383.0,13056.0,15792.0,489.0,8939.0,4324.0,...,7836,8344.0,3855.0,4489.0,7589.0,4500.0,3089.0,6707.0,5209.0,1498.0
2020,297,25,12,27166,2386.0,13092.0,15811.0,489.0,8963.0,4335.0,...,7871,8379.0,3872.0,4507.0,7624.0,4516.0,3108.0,6726.0,5224.0,1502.0
2020,298,26,12,27308,2389.0,13137.0,15893.0,492.0,9024.0,4390.0,...,7912,8442.0,3898.0,4544.0,7663.0,4539.0,3124.0,6754.0,5245.0,1509.0
2020,299,27,12,27453,2393.0,13164.0,15914.0,493.0,9057.0,4406.0,...,7952,8477.0,3915.0,4562.0,7703.0,4559.0,3144.0,6787.0,5267.0,1520.0
2020,300,28,12,27914,2470.0,13332.0,16171.0,497.0,9261.0,4569.0,...,8105,8684.0,4022.0,4662.0,7806.0,4612.0,3194.0,6869.0,5327.0,1542.0
2020,301,29,12,28398,2551.0,13502.0,16446.0,501.0,9535.0,4715.0,...,8313,8927.0,4139.0,4787.0,7956.0,4703.0,3253.0,6967.0,5401.0,1566.0
2020,302,30,12,28924,2566.0,13713.0,16854.0,502.0,9787.0,4835.0,...,8516,9143.0,4240.0,4902.0,8099.0,4779.0,3320.0,7022.0,5443.0,1579.0


In [55]:
df.shape

(303, 43)

In [47]:
df.dtypes

dayFromStart            int64
day                     int64
month                   int64
LJ regija               int64
po regija             float64
kr regija             float64
ce regija             float64
foreign regija        float64
nm regija             float64
kk regija             float64
mb regija             float64
ms regija             float64
za regija             float64
ng regija             float64
kp regija             float64
tested_poz_today        int64
tested_poz_to_date      int64
tested_today            int64
tested_to_date          int64
age 15-24             float64
age 15-24 FEMALE      float64
age 15-24 MEN         float64
age 25-34             float64
age 25-34 FEMALE      float64
age 25-34 MEN         float64
age 35-44             float64
age 35-44 FEMALE      float64
age 35-44 MEN         float64
age 45-54             float64
age 45-54 FEMALE      float64
age 45-54 MEN         float64
age 55-64               int64
age 55-64 FEMALE      float64
age 55-64 

In [49]:
df.isnull().sum()

dayFromStart          0
day                   0
month                 0
LJ regija             0
po regija             0
kr regija             0
ce regija             0
foreign regija        0
nm regija             0
kk regija             0
mb regija             0
ms regija             0
za regija             0
ng regija             0
kp regija             0
tested_poz_today      0
tested_poz_to_date    0
tested_today          0
tested_to_date        0
age 15-24             0
age 15-24 FEMALE      0
age 15-24 MEN         0
age 25-34             0
age 25-34 FEMALE      0
age 25-34 MEN         0
age 35-44             0
age 35-44 FEMALE      0
age 35-44 MEN         0
age 45-54             0
age 45-54 FEMALE      0
age 45-54 MEN         0
age 55-64             0
age 55-64 FEMALE      0
age 55-64 MEN         0
age 65-74             0
age 65-74 FEMALE      0
age 65-74 MEN         0
age 75-84             0
age 75-84 FEMALE      0
age 75-84 MEN         0
age 85+               0
age 85+ FEMALE  

In [50]:
df.fillna(0,inplace=True)


In [51]:
df.isnull().sum()

dayFromStart          0
day                   0
month                 0
LJ regija             0
po regija             0
kr regija             0
ce regija             0
foreign regija        0
nm regija             0
kk regija             0
mb regija             0
ms regija             0
za regija             0
ng regija             0
kp regija             0
tested_poz_today      0
tested_poz_to_date    0
tested_today          0
tested_to_date        0
age 15-24             0
age 15-24 FEMALE      0
age 15-24 MEN         0
age 25-34             0
age 25-34 FEMALE      0
age 25-34 MEN         0
age 35-44             0
age 35-44 FEMALE      0
age 35-44 MEN         0
age 45-54             0
age 45-54 FEMALE      0
age 45-54 MEN         0
age 55-64             0
age 55-64 FEMALE      0
age 55-64 MEN         0
age 65-74             0
age 65-74 FEMALE      0
age 65-74 MEN         0
age 75-84             0
age 75-84 FEMALE      0
age 75-84 MEN         0
age 85+               0
age 85+ FEMALE  

In [52]:
df.head(10)

,dayFromStart,day,month,LJ regija,po regija,kr regija,ce regija,foreign regija,nm regija,kk regija,...,age 55-64 MEN,age 65-74,age 65-74 FEMALE,age 65-74 MEN,age 75-84,age 75-84 FEMALE,age 75-84 MEN,age 85+,age 85+ FEMALE,age 85+ MEN
year,,,,,,,,,,,,,,,,,,,,,
2020,1,4,3,1,0.0,0.0,0.0,0.0,0.0,0.0,...,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2020,2,5,3,5,0.0,0.0,0.0,0.0,0.0,0.0,...,1,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
2020,3,6,3,7,0.0,0.0,0.0,0.0,1.0,0.0,...,3,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
2020,4,7,3,8,0.0,0.0,0.0,0.0,3.0,0.0,...,3,2.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
2020,5,8,3,10,0.0,0.0,0.0,0.0,6.0,0.0,...,4,3.0,1.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0
2020,6,9,3,16,0.0,0.0,0.0,0.0,11.0,0.0,...,5,5.0,1.0,4.0,0.0,0.0,0.0,1.0,1.0,0.0
2020,7,10,3,27,2.0,0.0,0.0,0.0,12.0,0.0,...,7,6.0,1.0,5.0,0.0,0.0,0.0,2.0,1.0,1.0
2020,8,11,3,44,3.0,1.0,4.0,0.0,18.0,0.0,...,9,8.0,2.0,6.0,1.0,1.0,0.0,3.0,1.0,2.0
2020,9,12,3,63,3.0,2.0,13.0,1.0,30.0,1.0,...,15,8.0,2.0,6.0,1.0,1.0,0.0,5.0,2.0,3.0
